In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [2]:
from utils import loader
from utils import debugger 
from utils import preprocessing as pp 
from utils import misc
from utils import constant
import implementations as impl
import cost
import model

In [3]:
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, x, ids = loader.load_csv_data(DATA_TRAIN_PATH)

In [93]:
to_replace = [(constant.UNDEF_VAL, 'most_frequent')]

augment_param = {
    'degrees': [2],
    'add_bias' : True,
    'add_cross': True
}

import time

start = time.time()


x_split, y_split = pp.preprocess_jet_num(x, y, to_replace, do_normalise=False, augment_param=augment_param)

end = time.time()
print(end - start)

85.25590682029724


In [94]:
x_split[2].shape, y_split[2].shape

((72543, 465), (72543,))

In [95]:
data_part = []

nb_samples = []
for x_, y_ in zip(x_split, y_split):
    train_data, test_data = misc.random_data_split(y_, x_, k_fold=10)
    data = {'train': train_data, 'test': test_data}
    data_part.append(data)
    nb_samples.append(len(test_data[1]))

In [99]:
models = []
learning_param = { 'lambda_': 1e-6 }

acc_list = []

for idx in range(len(data_part)):
    x_tr, y_tr = data_part[idx]['train']
    nb_features = x_tr.shape[1]
    
    w_initial = np.random.uniform(0,1,size=nb_features) 
    model_ls = model.Model('ridge_regression', w_initial, learning_param, debug=True)
    
    model_ls.learn_weights(y_tr, x_tr)
    models.append(model_ls)
    
    tx_te, y_te = data_part[idx]['test']
    y_ = model_ls.predict(tx_te)
    acc_list.append(misc.accuracy(y_te, y_))

learning weights...
done.
learning weights...
done.
learning weights...
done.


In [100]:
acc_list

[0.8286457812030827, 0.7704410626773278, 0.7823269920044114]

In [101]:
avg_acc = np.dot(acc_list, nb_samples) / sum(nb_samples)
avg_acc

0.7971518860754431

In [102]:
# Save weights
weights = [trained_model.w for trained_model in models]
file_name = 'weights.npy'
np.save(file_name, weights) # save

In [103]:
weights[2].shape

(465,)

In [104]:
weights_from_file = np.load(file_name) # load

In [105]:
weights_from_file[0].shape

(210,)